# هدف : تشخیص انگشتان دست

# وارد کردن کتابخانه های لازم

In [2]:
import cv2 
import numpy as np 
import mediapipe as mp
from datetime import datetime

2024-08-25 23:17:52.571716: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-25 23:17:52.581161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-25 23:17:52.594391: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-25 23:17:52.597557: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-25 23:17:52.607117: I tensorflow/core/platform/cpu_feature_guar

# نادیده گرفتن هشدار های کم اهیمت

In [3]:
import warnings
from google.protobuf import message as pb_message

warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database')

# تعریف رنگ ها

In [4]:
# opencv تعریف رنگ‌ها برای استفاده در ترسیمات البته  به صورت برعکس  چون 
red = (0, 0, 255)
green = (0, 255, 0)
blue = (255, 0, 0)

# 1-  Mediapipe راه‌اندازی مدل تشخیص دست از

In [5]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)

# static_image_mode : برای پردازش ویدئو استفاده می‌شود (False) پیشفرض
# True : مدل هر فریم را به‌عنوان یک تصویر مستقل در نظر می‌گیرد و دست‌ها را در هر فریم مجدداً شناسایی می‌کند. این حالت برای پردازش تصاویر ثابت مناسب است
# False : اگر از یک ویدئو استفاده می‌کنید و می‌خواهید شناسایی دست‌ها در فریم‌های بعدی نیز انجام شود (بدون شناسایی مجدد در هر فریم)

# max_num_hands : حداکثر تعداد شناسایی دست ها
# 1 
# 2 (پیشفرض)

# min_detection_confidence: فقط دست‌هایی که با اطمینان از آن درصد  یا بیشتر شناسایی می‌شوند، در نظر گرفته خواهند شد
#  برای شناسایی دست‌ها. مقادیر بین 0.0 تا 1.0 قابل قبول هستند (confidence)  حداقل اطمینان 
# مقدار پیش‌فرض: 0.5
# مقدار بالاتر از 0.5 پیشنهاد می‌شود تا شناسایی دست‌ها دقیق‌تر باشد

# min_tracking_confidence : دست‌ها بعد از شناسایی اولیه مقادیر بین 0.0 تا 1.0 قابل قبول هستند (tracking) حداقل اطمینان برای دنبال‌کردن 
# مقدار پیش‌فرض: 0.5
# اگر نیاز دارید دست‌ها با اطمینان بیشتری در طول ویدئو دنبال شوند، این مقدار را افزایش دهید

I0000 00:00:1724615274.354204    5052 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1724615274.357040    5217 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)


# 2- Mediapipe راه‌اندازی ابزارهای ترسیم از

In [6]:
mp_drawing = mp.solutions.drawing_utils

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<img src = 'Data/mediapipe.png' width=400>

# حلقه برای استفاده از تصاویر وبکم

In [7]:
import time

# راه‌اندازی وبکم
cap = cv2.VideoCapture(0)

# تابع برای محاسبه فاصله بین دو نقطه
def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    # # RGB به BGR 
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # روی هر فریم MediaPipe پردازش تصویر با 
    # تعیین می شود x,y,z است که موقعیت آن‌ها بر اساس مختصات (landmarks) یک لیست از دست‌های شناسایی‌شده در فریم فعلی تصویر یا ویدیو را برمی‌گرداند. هر دست در این لیست شامل نقاط کلیدی
    results = hands.process(frame_rgb)
    
    # اگر حداقل یک دست شناسایی شده باشد
    if results.multi_hand_landmarks: 
        for id, hand_landmarks in enumerate(results.multi_hand_landmarks): 

            # لیست برای شمارش انگشتان باز
            fingers = []

            # بررسی جهت دست (راست یا چپ)
            hand_label = results.multi_handedness[id].classification[0].label

            # بررسی انگشت شست (نقاط ۲، ۳ و ۴)
            thumb_tip = hand_landmarks.landmark[4]  # نوک انگشت شست
            thumb_ip = hand_landmarks.landmark[3]   # بند وسط انگشت شست
            thumb_mcp = hand_landmarks.landmark[2]  # بند پایه انگشت شست
            index_mcp = hand_landmarks.landmark[7]  # بند وسط انگشت اشاره

            # محاسبه فاصله نوک شست با وسط بند انگشت اشاره برای مشت کردن
            thumb_index_distance = calculate_distance((thumb_tip.x, thumb_tip.y), (index_mcp.x, index_mcp.y))

            # بررسی اینکه نوک انگشت شست نسبت به بند پایه و وسط انگشت در دست راست و چپ کجاست
            if (hand_label == 'Right' and thumb_tip.x < thumb_mcp.x and thumb_index_distance > 0.1) or \
               (hand_label == 'Left' and thumb_tip.x > thumb_mcp.x and thumb_index_distance > 0.1):
                fingers.append(1)
            else:
                fingers.append(0)

            # بررسی سایر انگشتان (۸، ۱۲، ۱۶، ۲۰)
            for i in [8, 12, 16, 20]:
                finger_tip = hand_landmarks.landmark[i]
                finger_dip = hand_landmarks.landmark[i - 2]

                if finger_tip.y < finger_dip.y:  # نوک انگشت بالاتر از بند نزدیک‌تر به کف دست
                    fingers.append(1)
                else:
                    fingers.append(0)

            # نمایش تعداد انگشتان باز
            cv2.putText(frame, f'Hand {hand_label} Fingers: {sum(fingers)}', (10, 75 + id * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, blue, 2)
            
            # رسم نقاط کلیدی و اتصالات
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # نمایش تصویر
    cv2.imshow('Hand Tracking', frame)

    # به‌دست آوردن زمان کنونی
    current_time = datetime.now()
    current_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

    k = cv2.waitKey(1) & 0xFF 
    if k==27:    # برای خروج Esc کلید 
        break
    
    elif k== ord('q'): # q یا دکمه 
        break

    elif k == ord('s'):
        filename = f'Data/captured_image_{current_time}.jpg'
        cv2.imwrite(filename,frame)
        print(f"Image saved as {filename}")
    
cap.release()
cv2.destroyAllWindows()


W0000 00:00:1724615274.370755    5209 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1724615274.391774    5205 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# نمایش نتیجه

<img src = 'Data/captured_image_2024-08-24 13:22:48.jpg'>
<img src = 'Data/captured_image_2024-08-24 13:22:54.jpg'>
<img src = 'Data/captured_image_2024-08-24 13:22:57.jpg'>